# Powering Tetouan: Time Series Forecasting on Power Consumption in Quads, Smir and Boussafou zones.

In [25]:
import pandas as pd
import numpy as np


In [26]:
df = pd.read_csv('sample_data.csv')

In [27]:
df.head()

,Datetime,predicted_Zone1,predicted_Zone2,predicted_Zone3
0,2/12/2017 14:40,30000,10000,20000
1,11/22/2017 15:30,30000,10000,20000
2,10/15/2017 1:50,30000,10000,20000
3,9/11/2017 23:00,30000,10000,20000
4,12/26/2017 1:20,30000,10000,20000


In [45]:
df.shape

(10483, 4)

In [28]:


from datetime import datetime

df['Datetime'] = df['Datetime'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M"))



In [29]:
df['Datetime'].dt.hour

0        14
1        15
2         1
3        23
4         1
         ..
10478    13
10479    14
10480     7
10481    11
10482    23
Name: Datetime, Length: 10483, dtype: int64

In [30]:
df.isnull().sum()

Datetime           0
predicted_Zone1    0
predicted_Zone2    0
predicted_Zone3    0
dtype: int64

In [31]:
train = pd.read_csv('train_data.csv')

In [32]:
train.head()

,Datetime,Temperature,Humidity,WindSpeed,GeneralDiffuseFlows,DiffuseFlows,PowerConsumption_Zone1,PowerConsumption_Zone2,PowerConsumption_Zone3
0,5/5/2017 22:30,17.84,55.42,4.920,0.099,0.089,39168.00000,24375.23220,23675.46559
1,5/5/2017 21:30,18.71,49.68,4.919,0.048,0.130,43272.39344,25723.83901,24730.68826
2,4/11/2017 16:40,15.76,85.70,0.068,200.700,182.800,32489.12809,18084.31772,17716.36364
3,9/30/2017 21:40,19.19,83.60,4.919,0.073,0.122,40218.05310,23864.03326,19198.03882
4,1/5/2017 10:40,8.82,65.24,0.084,322.500,40.380,31552.40506,20739.20973,15226.98795


In [33]:
from datetime import datetime

train['Datetime'] = train['Datetime'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M"))



In [42]:
train.head(10)

,Datetime,Temperature,Humidity,WindSpeed,GeneralDiffuseFlows,DiffuseFlows,PowerConsumption_Zone1,PowerConsumption_Zone2,PowerConsumption_Zone3
0,2017-05-05 22:30:00,17.840,55.42,4.920,0.099,0.089,39168.00000,24375.23220,23675.465590
1,2017-05-05 21:30:00,18.710,49.68,4.919,0.048,0.130,43272.39344,25723.83901,24730.688260
2,2017-04-11 16:40:00,15.760,85.70,0.068,200.700,182.800,32489.12809,18084.31772,17716.363640
3,2017-09-30 21:40:00,19.190,83.60,4.919,0.073,0.122,40218.05310,23864.03326,19198.038820
4,2017-01-05 10:40:00,8.820,65.24,0.084,322.500,40.380,31552.40506,20739.20973,15226.987950
5,2017-02-25 16:40:00,16.840,37.18,0.082,375.700,397.800,31606.77966,18167.78116,16093.266330
6,2017-01-24 21:00:00,7.870,59.73,0.088,0.044,0.134,44153.92405,27195.13678,25405.301200
7,2017-12-23 13:20:00,16.770,67.52,0.077,236.100,164.200,31349.04943,24994.16999,10089.795920
8,2017-11-29 05:30:00,16.890,68.62,4.914,0.029,0.133,22756.92308,17910.74380,9929.638554
9,2017-12-10 07:30:00,5.768,87.70,0.086,0.073,0.152,18323.95437,13130.40810,6263.625450


In [41]:
train['Datetime'].dt.minute

0        30
1        30
2        40
3        40
4        40
         ..
41927    40
41928    50
41929    30
41930    40
41931    30
Name: Datetime, Length: 41932, dtype: int64

In [34]:
train.isnull().sum()

Datetime                  0
Temperature               0
Humidity                  0
WindSpeed                 0
GeneralDiffuseFlows       0
DiffuseFlows              0
PowerConsumption_Zone1    0
PowerConsumption_Zone2    0
PowerConsumption_Zone3    0
dtype: int64

In [35]:
test = pd.read_csv('test_data.csv')

In [44]:
train.shape

(41932, 9)

In [43]:
test.shape

(10483, 6)

In [37]:
test.isnull().sum()

Datetime               0
Temperature            0
Humidity               0
WindSpeed              0
GeneralDiffuseFlows    0
DiffuseFlows           0
dtype: int64

In [46]:
train.dtypes

Datetime                  datetime64[ns]
Temperature                      float64
Humidity                         float64
WindSpeed                        float64
GeneralDiffuseFlows              float64
DiffuseFlows                     float64
PowerConsumption_Zone1           float64
PowerConsumption_Zone2           float64
PowerConsumption_Zone3           float64
dtype: object